<a href="https://colab.research.google.com/github/gichurejoy/Machine-Learning/blob/main/stockanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Flask


In [ ]:
import pandas as pd
from datetime import datetime
from flask import Flask, request, jsonify
import os

app = Flask(__name__)

@app.route('/get_data/', methods=['POST'])
def get_data():
    try:

        data = request.get_json()
        startDate = data['startDate']
        endDate = data['endDate']
        branch = data['branch']
        itemCode = data['itemCode']

        dataset = pd.read_excel('/content/Sample Data S00062.xlsx')
        dataset['Document Date'] = pd.to_datetime(dataset['Document Date'], dayfirst=True)

        rs = dataset[(dataset['Document Date'] >= startDate) & (
            dataset['Document Date'] < endDate) & (dataset['Branch'] == branch) & (dataset['Item Code'] == itemCode)]
        rs = rs.sort_values(by='Document Date')

        rs['RUNNING STOCK'] = rs['Total Qty IN'].cumsum() - rs['Total Qty OUT'].cumsum()
        rs['DAYS BETWEEN'] = rs['Document Date'].diff().dt.days

        filtered_rs = rs[rs['RUNNING STOCK'] <= 0]

        sum_days_between = filtered_rs['DAYS BETWEEN'].sum()
        result = rs.to_dict(orient='records')

        ctime = datetime.now()
        output_file = f'{itemCode}_output_data{ctime}.xlsx'
        rs.to_excel(output_file)


        data_to_return = filtered_rs.to_dict(orient='records')
        return jsonify({
            "File Upload": output_file,
            "Out OF Stock Days": float(sum_days_between),
            "data": data_to_return
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8001)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8001
 * Running on http://172.28.0.12:8001
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [19/Oct/2023 07:47:37] "POST /get_data/ HTTP/1.1" 200 -


In [50]:
!pip install pyngrok
from pyngrok import ngrok

port = 8001

public_url = ngrok.connect(addr=port)

print('ngrok is active at:', public_url)


ngrok is active at: NgrokTunnel: "https://760c-35-234-19-68.ngrok.io" -> "http://localhost:8001"


In [30]:
with open('/content/response.json', 'r') as json_file:
    json_data = json.load(json_file)
    print(json_data)


{'File Upload': 'SAL009_output_data2023-10-18 09:01:35.262099.xlsx', 'Out OF Stock Days': 120.0, 'data': [{'Branch': 'DAN002', 'DAYS BETWEEN': 1.0, 'Date Day': 5, 'Date Month': 1, 'Date Year': 2023, 'Document Date': 'Thu, 05 Jan 2023 00:00:00 GMT', 'Item Code': 'SAL009', 'Item Name': 'SALIT COOKING OIL 20LT', 'RUNNING STOCK': 0.0, 'Total Qty IN': 35.0, 'Total Qty OUT': 1692.0}, {'Branch': 'DAN002', 'DAYS BETWEEN': 1.0, 'Date Day': 6, 'Date Month': 1, 'Date Year': 2023, 'Document Date': 'Fri, 06 Jan 2023 00:00:00 GMT', 'Item Code': 'SAL009', 'Item Name': 'SALIT COOKING OIL 20LT', 'RUNNING STOCK': 0.0, 'Total Qty IN': 26.0, 'Total Qty OUT': 26.0}, {'Branch': 'DAN002', 'DAYS BETWEEN': 1.0, 'Date Day': 7, 'Date Month': 1, 'Date Year': 2023, 'Document Date': 'Sat, 07 Jan 2023 00:00:00 GMT', 'Item Code': 'SAL009', 'Item Name': 'SALIT COOKING OIL 20LT', 'RUNNING STOCK': 0.0, 'Total Qty IN': 52.0, 'Total Qty OUT': 52.0}, {'Branch': 'DAN002', 'DAYS BETWEEN': 1.0, 'Date Day': 11, 'Date Month': 1

In [47]:
from IPython.display import HTML
import json

chart_js_code = """
<script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
<canvas id="real-time-chart"></canvas>
<script>
    function fetch_data_from_local_json() {
        try {
            // You can specify the path to your JSON file here
            var json_file_path = 'response.json'

            // Perform an AJAX request to fetch the JSON data
            var xhr = new XMLHttpRequest();
            xhr.open('GET', json_file_path, false);  // Synchronous request
            xhr.send(null);

            if (xhr.status === 200) {
                return JSON.parse(xhr.responseText);
            }
        } catch (e) {
            console.error("Error:", e);
        }
        return null;
    }

    var ctx = document.getElementById('real-time-chart').getContext('2d');
    var data = {
        labels: [],
        datasets: [
            {
                label: 'Total Qty IN',
                borderColor: 'blue',
                data: [],
            },
            {
                label: 'Total Qty OUT',
                borderColor: 'red',
                data: [],
            },
            {
                label: 'Running Stock',
                borderColor: 'green',
                data: [],
            }
        ],
    };
    var config = {
        type: 'line',
        data: data,
        options: {
            scales: {
                x: {
                    type: 'linear',
                    position: 'bottom',
                },
                y: {
                    beginAtZero: true,
                },
            },
        },
    };
    var chart = new Chart(ctx, config);

    function updateChart() {
        var newData = fetch_data_from_local_json();
        if (newData) {
            data.labels.push(newData['data'][0]['Document Date']);
            data.datasets[0].data.push(newData['data'][0]['Total Qty IN']);
            data.datasets[1].data.push(newData['data'][0]['Total Qty OUT']);
            data.datasets[2].data.push(newData['data'][0]['RUNNING STOCK']);
            chart.update();
        }
    }

    // Update the chart every 5 seconds
    setInterval(updateChart, 5000);
</script>
"""

display(HTML(chart_js_code))
